In [1]:
import os 

In [2]:
%pwd

'/Users/xulitong/Desktop/nlpproject/end-to-end-text-summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/xulitong/Desktop/nlpproject/end-to-end-text-summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [7]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, creat_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH  
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        creat_directories([self.config.artifacts_root])
    

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_transformation

        creat_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        
        return data_transformation_config


In [9]:
import os
from src.TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config=config
        self.tokenizer= AutoTokenizer.from_pretrained(config.tokenizer_name)


    def convert_examples_to_features(self, example_batch):
        input_encoding=self.tokenizer(example_batch["dialogue"],
                                      max_length=1024,
                                      truncation=True)
        
        target_encoding=self.tokenizer(text_target=example_batch["summary"],
                                       max_length=128,
                                       truncation=True)
        
        return {
            "input_ids": input_encoding["input_ids"],
            "attention_mask": input_encoding["attention_mask"],
            "labels": target_encoding["input_ids"]
        }


    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [11]:
try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.convert()
    
except Exception as e:
    raise e 

[2024-09-24 16:51:19,965: INFO: common: yaml file: config/config.yaml successfully loaded]
[2024-09-24 16:51:19,970: INFO: common: yaml file: params.yaml successfully loaded]
[2024-09-24 16:51:19,971: INFO: common: Directory created at: artifacts]
[2024-09-24 16:51:19,971: INFO: common: Directory created at: artifacts/data_transformation]


/Users/xulitong/Desktop/nlpproject/end-to-end-text-summarizer/env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 101312.29 examples/s]
